## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,66.20,72,75,10.36,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,79.29,73,5,12.08,clear sky
2,2,Albany,US,42.6001,-73.9662,34.00,65,100,7.00,overcast clouds
3,3,Maniitsoq,GL,65.4167,-52.9000,2.62,81,7,6.11,clear sky
4,4,Hervey Bay,AU,-25.2986,152.8535,75.00,81,25,8.01,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 75


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,66.20,72,75,10.36,broken clouds
4,4,Hervey Bay,AU,-25.2986,152.8535,75.00,81,25,8.01,scattered clouds
6,6,Brandfort,ZA,-28.7001,26.4597,59.00,82,0,5.75,clear sky
7,7,Lebu,CL,-37.6167,-73.6500,57.54,75,0,12.62,clear sky
10,10,Launceston,AU,-41.4500,147.1667,68.00,63,40,5.75,scattered clouds
18,18,Busselton,AU,-33.6500,115.3333,60.01,98,0,6.08,clear sky
21,21,Saint George,US,37.1041,-113.5841,55.00,22,75,9.22,broken clouds
24,24,Port Elizabeth,ZA,-33.9180,25.5701,66.20,68,0,11.50,clear sky
25,25,Coquimbo,CL,-29.9533,-71.3436,66.20,63,0,9.22,clear sky
27,27,Ngama,TD,11.7833,17.1667,69.80,23,0,12.84,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                226
City                   226
Country                218
Lat                    226
Lng                    226
Max Temp               226
Humidity               226
Cloudiness             226
Wind Speed             226
Current Description    226
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,66.20,72,75,10.36,broken clouds
4,4,Hervey Bay,AU,-25.2986,152.8535,75.00,81,25,8.01,scattered clouds
6,6,Brandfort,ZA,-28.7001,26.4597,59.00,82,0,5.75,clear sky
7,7,Lebu,CL,-37.6167,-73.6500,57.54,75,0,12.62,clear sky
10,10,Launceston,AU,-41.4500,147.1667,68.00,63,40,5.75,scattered clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bredasdorp,ZA,66.20,broken clouds,-34.5322,20.0403,
4,Hervey Bay,AU,75.00,scattered clouds,-25.2986,152.8535,
6,Brandfort,ZA,59.00,clear sky,-28.7001,26.4597,
7,Lebu,CL,57.54,clear sky,-37.6167,-73.6500,
10,Launceston,AU,68.00,scattered clouds,-41.4500,147.1667,
18,Busselton,AU,60.01,clear sky,-33.6500,115.3333,
21,Saint George,US,55.00,broken clouds,37.1041,-113.5841,
24,Port Elizabeth,ZA,66.20,clear sky,-33.9180,25.5701,
25,Coquimbo,CL,66.20,clear sky,-29.9533,-71.3436,
27,Ngama,TD,69.80,clear sky,11.7833,17.1667,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bredasdorp,ZA,66.20,broken clouds,-34.5322,20.0403,Bredasdorp Country Manor
4,Hervey Bay,AU,75.00,scattered clouds,-25.2986,152.8535,Shelly Bay Resort
6,Brandfort,ZA,59.00,clear sky,-28.7001,26.4597,SUMFRA GUESTHOUSE
7,Lebu,CL,57.54,clear sky,-37.6167,-73.6500,Hostal Las Lilas
10,Launceston,AU,68.00,scattered clouds,-41.4500,147.1667,Hotel Grand Chancellor Launceston


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))